In [ ]:
import requests
import json
from time import sleep
import csv
import math

# function to write results to a file
def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()

In [ ]:
table = []
otherNameList = []

# use the API to search for people associated with Vanderbilt University
# First search is for only one record, just to get the number of hits found
searchUri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start=1&rows=1'
response = requests.get(searchUri, headers={'Accept' : 'application/json'})
data = response.json()

In [ ]:
numberResults = data["num-found"]
print(data["num-found"])
numberPages = math.floor(numberResults/100)
print(numberPages)
remainder = numberResults - 100*numberPages
print(remainder)

In [ ]:
pageCount = 0
# use the API to search for people associated with Vanderbilt University
print('page: ', pageCount)
searchUri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start='+str(pageCount*100+1)
response = requests.get(searchUri, headers={'Accept' : 'application/json'})
print(response.url)
data = response.json()
orcidsDictsList = data['result']

In [ ]:
# extract the identifier strings from the data structure
orcids = []
for orcidDict in orcidsDictsList:
    dictionary = {'id': orcidDict['orcid-identifier']['path'], 'iri': orcidDict['orcid-identifier']['uri']}
    orcids.append(dictionary)

In [ ]:
response = requests.get(orcids[0]['iri'], headers={'Accept' : 'application/json'})
data = response.json()

In [ ]:
#print(json.dumps(data, indent = 2))
orcidId = data['orcid-identifier']['path']
#print(orcidId)
# if there isn't a name, then go on to the next ORCID
if data['person']['name']['given-names']:  
    givenNames = data['person']['name']['given-names']['value']
else:
    givenNames = ''
if data['person']['name']['family-name']:
    familyName = data['person']['name']['family-name']['value']
else:
    familyName = ''
#print(givenNames, ' ', familyName)
otherNames = data['person']['other-names']['other-name']
for otherName in otherNames:
    #print(otherName['content'])
    otherNameList.append([orcidId, otherName['content']])


In [ ]:
affiliations = data['activities-summary']['employments']['affiliation-group']
#print(json.dumps(affiliations, indent = 2))
for affiliation in affiliations:
    summaries = affiliation['summaries']
    #print(summaries)
    #print()
    for summary in summaries:
        employment = summary['employment-summary']
        #print(json.dumps(employment, indent = 2))
        startDate = ''
        if employment['start-date']:
            if employment['start-date']['year']:
                startDate += employment['start-date']['year']['value']
                startMonth = employment['start-date']['month']
                if startMonth:
                    startDate += '-' + startMonth['value']
                    startDay = employment['start-date']['day']
                    if startDay:
                        startDate += '-' + startDay['value']
        #print('start date: ', startDate)
        endDate = ''
        if employment['end-date']:
            if employment['end-date']['year']:
                endDate += employment['end-date']['year']['value']
                endMonth = employment['end-date']['month']
                if endMonth:
                    endDate += '-' + endMonth['value']
                    endDay = employment['end-date']['day']
                    if endDay:
                        endDate += '-' + endDay['value']
        #print('end date: ', endDate)
        department = employment['department-name']
        # if there is no value for department, set it to empty string
        if not department:
            department = ''
        #print(department)
        if employment['organization']:
            organization = employment['organization']['name']
        #print(organization)
        if 'Vanderbilt University' in organization:
            print(orcidId, givenNames, familyName, startDate, endDate, department, organization)
            table.append([orcidId, givenNames, familyName, startDate, endDate, department, organization])
        print(table)
sleep(.25)

In [ ]:
print(table)

fileName = 'orcid_data.csv'
writeCsv(fileName, table)
fileName = 'orcid_other_names.csv'
writeCsv(fileName, otherNameList)


In [ ]:
import requests
import json
from time import sleep
import csv
import math

# function to write results to a file
def writeCsv(fileName, array):
    fileObject = open(fileName, 'w', newline='', encoding='utf-8')
    writerObject = csv.writer(fileObject)
    for row in array:
        writerObject.writerow(row)
    fileObject.close()
    
table = []
otherNameList = []

# use the API to search for people associated with Vanderbilt University
# First search is for only one record, just to get the number of hits found
searchUri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start=1&rows=1'
response = requests.get(searchUri, headers={'Accept' : 'application/json'})
data = response.json()
numberResults = data["num-found"]
#print(data["num-found"])
numberPages = math.floor(numberResults/100)
#print(numberPages)
remainder = numberResults - 100*numberPages
#print(remainder)

for pageCount in range(0, numberPages+1):  # the remainder will be caught when pageCount = numberPages
    print('page: ', pageCount)
    searchUri = 'https://pub.orcid.org/v2.0/search/?q=affiliation-org-name:"Vanderbilt+University"&start='+str(pageCount*100+1)
    response = requests.get(searchUri, headers={'Accept' : 'application/json'})
    print(response.url)
    data = response.json()
    orcidsDictsList = data['result']

    # extract the identifier strings from the data structure
    orcids = []
    for orcidDict in orcidsDictsList:
        dictionary = {'id': orcidDict['orcid-identifier']['path'], 'iri': orcidDict['orcid-identifier']['uri']}
        orcids.append(dictionary)

    for orchidIndex in range(0, len(orcids)):
        response = requests.get(orcids[orchidIndex]['iri'], headers={'Accept' : 'application/json'})
        data = response.json()

        #print(json.dumps(data, indent = 2))
        orcidId = data['orcid-identifier']['path']
        #print(orcidId)
        # if there isn't a name, then go on to the next ORCID
        if not data['person']['name']:
            continue
        if data['person']['name']['given-names']:  
            givenNames = data['person']['name']['given-names']['value']
        else:
            givenNames = ''
        if data['person']['name']['family-name']:
            familyName = data['person']['name']['family-name']['value']
        else:
            familyName = ''
        #print(givenNames, ' ', familyName)
        otherNames = data['person']['other-names']['other-name']
        for otherName in otherNames:
            #print(otherName['content'])
            otherNameList.append([orcidId, otherName['content']])

        affiliations = data['activities-summary']['employments']['affiliation-group']
        #print(json.dumps(affiliations, indent = 2))
        for affiliation in affiliations:
            summaries = affiliation['summaries']
            #print(summaries)
            #print()
            for summary in summaries:
                employment = summary['employment-summary']
                #print(json.dumps(employment, indent = 2))
                startDate = ''
                if employment['start-date']:
                    if employment['start-date']['year']:
                        startDate += employment['start-date']['year']['value']
                        startMonth = employment['start-date']['month']
                        if startMonth:
                            startDate += '-' + startMonth['value']
                            startDay = employment['start-date']['day']
                            if startDay:
                                startDate += '-' + startDay['value']
                #print('start date: ', startDate)
                endDate = ''
                if employment['end-date']:
                    if employment['end-date']['year']:
                        endDate += employment['end-date']['year']['value']
                        endMonth = employment['end-date']['month']
                        if endMonth:
                            endDate += '-' + endMonth['value']
                            endDay = employment['end-date']['day']
                            if endDay:
                                endDate += '-' + endDay['value']
                #print('end date: ', endDate)
                department = employment['department-name']
                # if there is no value for department, set it to empty string
                if not department:
                    department = ''
                #print(department)
                if employment['organization']:
                    organization = employment['organization']['name']
                #print(organization)
                if 'Vanderbilt University' in organization:
                    print(orcidId, givenNames, familyName, startDate, endDate, department, organization)
                    table.append([orcidId, givenNames, familyName, startDate, endDate, department, organization])
                #print(table)
        sleep(.25)

print()
print('Done')
fileName = 'orcid_data.csv'
writeCsv(fileName, table)
fileName = 'orcid_other_names.csv'
writeCsv(fileName, otherNameList)
